# Data Wrangling - Potholes Dataset

Thibautl Dody
08/11/2017

The purpose of this notebook is to describe the process used to verify the quality of the potholes data set. The output of this notebook is a new csv file **Closed_Pothole_Cases_Cleaned.csv** located in *./Cleaned Data/* folder.

Data source: https://data.cityofboston.gov/City-Services/Requests-for-Pothole-Repair/n65p-xaz7/data
The data is filtered down to the requests made after January 1st 2014.

## 1. Process

The first step consists of a visual inspection of the csv file (loaded using excel). This phase is critical as it is used to define how the data will be imported and how the parameters of the import will be defined. Once the data has been imported and investigated, it needs to be cleaned and re-organized. To do so, missing values are either dropped or estimated and unrelevant features are removed from the set.

The file **Closed_Pothole_Cases.csv** is located in *./Original Data/* folder.

In [30]:
# Import all the python libraries needed for the wrangling
import numpy as np
import pandas as pd

## 2. Visual inspection

The dataset contains has the following properties that will impact the import:
- The file contains missing data
- The file contains date and times (OPEN_DT, CLOSED_DT)
- The file contains columns that are empty

In order to facilitate the filtering of the dataset, the choice is made to import the entire content of the file and to modify the dataset using Pandas' tools.

In [31]:
# Import the data as Pandas dataframe
fileFullPath = "./Original Data/Closed_Pothole_Cases.csv"
potholes_raw_df = pd.read_csv(fileFullPath, parse_dates=[1,2,3])

Before diving into the content of the dataframe, the size and data type of the dataset are obtained.

In [32]:
# Dataframe Shape
potholes_raw_df.shape

(29451, 33)

In [33]:
# Dataframe Info
potholes_raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29451 entries, 0 to 29450
Data columns (total 33 columns):
CASE_ENQUIRY_ID                   29451 non-null int64
OPEN_DT                           29451 non-null datetime64[ns]
TARGET_DT                         29451 non-null datetime64[ns]
CLOSED_DT                         29118 non-null datetime64[ns]
OnTime_Status                     29450 non-null object
CASE_STATUS                       29451 non-null object
CLOSURE_REASON                    29125 non-null object
CASE_TITLE                        29451 non-null object
SUBJECT                           29451 non-null object
REASON                            29451 non-null object
TYPE                              29451 non-null object
QUEUE                             29451 non-null object
Department                        29451 non-null object
SubmittedPhoto                    16533 non-null object
ClosedPhoto                       11074 non-null object
Location                    

The features presented in the file are defined as:
 - *CASE_ENQUIRY_ID*: Case number assigned to the request to repair the pothole.
 - *OPEN_DT*: Date and time of the repair request.
 - *TARGET_DT*: Scheduled time for repair.
 - *CLOSED_DT*: Date and time the case was closed.
 - *OnTime_Status*: ONTIME if *CLOSED_DT*>TARGET_DT
 - *CASE_STATUS*: Case status.
 - *CLOSURE_REASON*: Reason for the case closure.
 - *CASE_TITLE*: Request type. In this case, the type is "Request for Pothole Repair".
 - *SUBJECT*: The city department in charge of the request.
 - *REASON*: Reason for the case opening.
 - *TYPE*: Specific reason for the case opening. In this case, the type is "Request for Pothole Repair".
 - *QUEUE*: Code corresponding to the department per neighborhood in charge of the repair.
 - *Department*: Code corresponding to the department in charge of the repair.
 - *SubmittedPhoto*: URL of the photo taken to support the claim.
 - *ClosedPhoto*:  URL of the photo taken to support the repair.
 - *Location*: Address of the pothole.
 - *fire_district*: Fire district corresponding to the pothole location.
 - *pwd_district*: Public Work district corresponding to the pothole location.
 - *city_council_district*: City Council district corresponding to the pothole location.
 - *police_district*: Police district corresponding to the pothole location.
 - *neighborhood*: Neighborhood corresponding to the pothole location.
 - *neighborhood_services_district*: Neighborhood Services district corresponding to the pothole location.
 - *ward*: Ward corresponding to the pothole location.
 - *precinct*: Precinct corresponding to the pothole location.
 - *land_usage*: Blank column.
 - *LOCATION_STREET_NAME*: Street number and street name corresponding to the pothole location.
 - *LOCATION_ZIPCODE*: Zip code corresponding to the pothole location.
 - *Property_Type*: Blank column.
 - *Property_ID*: Blank column.
 - *LATITUDE*: Latitude of the pothole location.
 - *LONGITUDE*: Longitude of the pothole location.
 - *Source*: Source of the request.
 - *Geocoded_Location*: Blank column

# 3. Feature cleaning

The following choices are made:
- Delete *land_usage*, *Property_Type*, *Property_ID*, and *Geocoded_Location* since they do not contain any data.
- Import *OPEN_DT* and *CLOSED_DT* as DateTime objects

In [34]:
# Defined the feature names to be deleted.
columnNameToDelete = ['land_usage','Property_Type','Property_ID','Geocoded_Location']

# Clone the dataset and delete features of the clone.
potholes_df = potholes_raw_df.copy()
potholes_df.drop(columnNameToDelete,inplace=True,axis=1)

potholes_df.shape

(29451, 29)

After checking the top records of the dataset, it seems that several columns are filled with the same values. This is due to the fact that the database is a subset of a larger one containing all the 311 calls to the city of Boston. We check that our assumption is correct by looking at the number of unique values in each column.

In [35]:
# Obtain the count of all and unique values for the entiere dataframe.
uniqueValuesCount_dict = {function.__name__:potholes_df.apply(function)
                          for function in (pd.Series.nunique, pd.Series.count)}
pd.concat(uniqueValuesCount_dict, axis=1)

,count,nunique
CASE_ENQUIRY_ID,29451,29451
OPEN_DT,29451,29381
TARGET_DT,29451,22068
CLOSED_DT,29118,29073
OnTime_Status,29450,2
CASE_STATUS,29451,2
CLOSURE_REASON,29125,18033
CASE_TITLE,29451,46
SUBJECT,29451,1
REASON,29451,1


After inspection of the results, the following decisions are made:
- *SUBJECT*, *REASON*, *TYPE*, and *Department* can be deleted

In [36]:
# The case status is dropped
potholes_df.drop(["SUBJECT","REASON","TYPE","Department"],inplace=True,axis=1)
potholes_df.shape

(29451, 25)

## 4. Feature conversion

### 4.1 Photos

In order to simplify the analysis and keep only relevant infomation, the *Submitted_Photo* and *Closed_Photo* features are converted into booleans. If the record contains the url of a picture, the value is converted to True.

In [37]:
# Create the new faeture and delete the ones containing the urls.
potholes_df["SubmittedPhoto_Bool"] = potholes_df["SubmittedPhoto"].notnull()
potholes_df["ClosedPhoto_Bool"] = potholes_df["ClosedPhoto"].notnull()
potholes_df.drop(["SubmittedPhoto","ClosedPhoto"],inplace=True,axis=1)
potholes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29451 entries, 0 to 29450
Data columns (total 25 columns):
CASE_ENQUIRY_ID                   29451 non-null int64
OPEN_DT                           29451 non-null datetime64[ns]
TARGET_DT                         29451 non-null datetime64[ns]
CLOSED_DT                         29118 non-null datetime64[ns]
OnTime_Status                     29450 non-null object
CASE_STATUS                       29451 non-null object
CLOSURE_REASON                    29125 non-null object
CASE_TITLE                        29451 non-null object
QUEUE                             29451 non-null object
Location                          29328 non-null object
fire_district                     29155 non-null float64
pwd_district                      29213 non-null object
city_council_district             29321 non-null float64
police_district                   29232 non-null object
neighborhood                      29228 non-null object
neighborhood_services_dist

### 4.2. Closure reason

Upon inspection of the *CLOSURE_REASON* column, a number of records contains information about the case. For instance,
- "Case Closed Case Invalid"
- "Case Closed Case Invalid  duplicate"
- "Case Closed Case Noted  No address given  please resubmit"
- "Case Closed Case Noted  no eform"  
  
In conlusion, it seems that the data needs to be cleaned to make sure only relevant records are considered.

The first step consists of identifying elements that are shared amongst all the relevant pothole case.
- *CLOSURE_REASON* contains the string "Case Resolved" => Case is acceptable
- *CLOSURE_REASON* contains the word "duplicate" => Case to be removed. (typos include duplcate, duplicte)
- *CLOSURE_REASON* contains the word "invalid" => Case to be removed.
- *CLOSURE_REASON* contains the words "better location" => Case to be removed.
- *CLOSURE_REASON* contains the words "please contact" => Case to be removed.
- *CLOSURE_REASON* contains the words "please call" => Case to be removed.
- *CLOSURE_REASON* contains the word "test" => Case to be removed.
- *CLOSURE_REASON* contains the words "could not find" => Case to be removed.
- *CLOSURE_REASON* contains the word "cannot" => Case to be removed.
- *CLOSURE_REASON* contains the word "private" => Case to be removed. (versions include prvt)
- *CLOSURE_REASON* contains the word "wrong" => Case to be removed.
- *CLOSURE_REASON* contains the word "nothing" => Case to be removed.
- *CLOSURE_REASON* contains the word "re-subnmit" => Case to be removed. (versions include resubmit)
- *CLOSURE_REASON* contains the words "no pot hole" => Case to be removed. (versions include no potholes, no sink hole)

Having one field set as a text box proved to be a challenge to deal with when cleaning the data.
Please note that afte the above filters are applied, roughly 400 claims are left unfiltered. Upon visual inspection, it seems that a large (if not all) requests contains relevant claim. The choice is made to keep the leftover in the set.

In [38]:
# Convert the CLOSURE_REASON content to lower case
potholes_df.CLOSURE_REASON = potholes_df.CLOSURE_REASON.str.lower()

# Create a list of all the cases considered as invalid
invalid_expressions = ['duplicate','duplcate','duplicte','invalid','better location','please contact',
                      'please call','test','could not find','cannot','private','prvt','wrong','nothing',
                      're-subnmit','resubmit','no pot hole','no potholes','no sink hole']

# In order to use the str.contains method, we need to replace the NaN values in the column by ''
potholes_df.CLOSURE_REASON.fillna(" ",inplace=True)
print(potholes_df.shape)

for invalid_key in invalid_expressions:
    dim_before = potholes_df.shape[0]
    potholes_df = potholes_df[~potholes_df.CLOSURE_REASON.str.contains(invalid_key)]
    dim_after = potholes_df.shape[0]
    print('Key: "'+invalid_key+'"',dim_before-dim_after,'matchs found.')

(29451, 25)
Key: "duplicate" 928 matchs found.
Key: "duplcate" 1 matchs found.
Key: "duplicte" 1 matchs found.
Key: "invalid" 392 matchs found.
Key: "better location" 167 matchs found.
Key: "please contact" 376 matchs found.
Key: "please call" 32 matchs found.
Key: "test" 18 matchs found.
Key: "could not find" 9 matchs found.
Key: "cannot" 16 matchs found.
Key: "private" 267 matchs found.
Key: "prvt" 18 matchs found.
Key: "wrong" 12 matchs found.
Key: "nothing" 43 matchs found.
Key: "re-subnmit" 0 matchs found.
Key: "resubmit" 42 matchs found.
Key: "no pot hole" 22 matchs found.
Key: "no potholes" 23 matchs found.
Key: "no sink hole" 3 matchs found.


### 4.2 Ward name

The first step consists of extracting the record without ward number and decide if they can be removed from the dataset.

In [39]:
print(potholes_df[potholes_df.ward.isnull()].shape)
potholes_df[potholes_df.ward.isnull()].head(50)

(105, 25)


,CASE_ENQUIRY_ID,OPEN_DT,TARGET_DT,CLOSED_DT,OnTime_Status,CASE_STATUS,CLOSURE_REASON,CASE_TITLE,QUEUE,Location,...,neighborhood_services_district,ward,precinct,LOCATION_STREET_NAME,LOCATION_ZIPCODE,LATITUDE,LONGITUDE,Source,SubmittedPhoto_Bool,ClosedPhoto_Bool
268,101001141766,2014-08-01 11:32:55,2014-08-05 11:32:55,NaT,OVERDUE,Open,,Request for Pothole Repair,PWDx_Roadway Repair_ARP_Resurfacing,NaN,...,NaN,NaN,NaN,NaN,NaN,42.3594,-71.0587,Self Service,False,False
584,101001130012,2014-07-16 10:19:52,2014-07-18 10:19:52,NaT,OVERDUE,Open,,Request for Pothole Repair,PWDx_Roadway Repair_CRP_Resurfacing,NaN,...,NaN,NaN,NaN,NaN,NaN,42.3594,-71.0587,Self Service,False,False
934,101001162784,2014-09-02 21:37:16,2015-09-05 08:30:00,NaT,OVERDUE,Open,,Request for Pothole Repair,PWDx_Roadway Repair_Ponding,NaN,...,NaN,NaN,NaN,NaN,NaN,42.3594,-71.0587,Self Service,False,False
1344,101001151755,2014-08-17 16:15:59,2014-08-20 08:30:00,2014-08-21 10:27:18,OVERDUE,Closed,case closed case resolved it belonngs to b w s...,Request for Pothole Repair,PWDx_District 05: South Boston,NaN,...,NaN,NaN,NaN,NaN,NaN,42.3594,-71.0587,Self Service,False,False
1398,101001031585,2014-02-20 11:15:12,2014-02-28 11:13:00,2014-02-27 12:08:36,ONTIME,Closed,case closed case resolved done,Request for Pothole Repair,PWDx_District 1A: Charlestown,NaN,...,NaN,NaN,NaN,NaN,NaN,42.3594,-71.0587,Employee Generated,False,False
1733,101001316714,2015-02-25 19:11:59,2015-02-27 08:30:00,2015-04-14 07:29:33,OVERDUE,Closed,case closed case resolved,Request for Pothole Repair,PWDx_District 10A: Roxbury,NaN,...,NaN,NaN,NaN,NaN,NaN,42.3594,-71.0587,Self Service,False,False
1989,101001353573,2015-04-10 11:50:15,2015-04-13 11:50:15,2015-04-13 12:49:32,OVERDUE,Closed,case closed case resolved completed from e.bro...,Request for Pothole Repair,PWDx_District 05: South Boston,NaN,...,NaN,NaN,NaN,NaN,NaN,42.3594,-71.0587,Self Service,False,True
2265,101001058326,2014-04-01 09:28:11,2014-04-03 09:28:11,2014-04-04 06:22:59,OVERDUE,Closed,case closed case resolved,Request for Pothole Repair,PWDx_District 05: South Boston,NaN,...,NaN,NaN,NaN,NaN,NaN,42.3594,-71.0587,Self Service,False,False
2456,101002047469,2017-03-23 18:36:11,2017-03-27 08:30:00,2017-03-24 06:50:36,ONTIME,Closed,case closed case noted no location specified ...,Request for Pothole Repair,PWDx_Requests for Pothole Repair,NaN,...,NaN,NaN,NaN,NaN,NaN,42.3594,-71.0587,Citizens Connect App,False,False
2766,101001338019,2015-03-20 09:18:21,2015-03-25 09:18:00,2015-03-24 10:51:36,ONTIME,Closed,case closed case resolved,Request for Pothole Repair,PWDx_District 10B: Roxbury,NaN,...,NaN,NaN,NaN,NaN,NaN,42.3594,-71.0587,Self Service,False,True


The records having both their location as Nan and their ward as Nan are removed. We choose to keep the other since there is a change to retrieve the locations later.

In [40]:
potholes_df = potholes_df[potholes_df.ward.notnull() & potholes_df.Location.notnull()]
potholes_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26976 entries, 0 to 29450
Data columns (total 25 columns):
CASE_ENQUIRY_ID                   26976 non-null int64
OPEN_DT                           26976 non-null datetime64[ns]
TARGET_DT                         26976 non-null datetime64[ns]
CLOSED_DT                         26648 non-null datetime64[ns]
OnTime_Status                     26976 non-null object
CASE_STATUS                       26976 non-null object
CLOSURE_REASON                    26976 non-null object
CASE_TITLE                        26976 non-null object
QUEUE                             26976 non-null object
Location                          26976 non-null object
fire_district                     26823 non-null float64
pwd_district                      26873 non-null object
city_council_district             26970 non-null float64
police_district                   26897 non-null object
neighborhood                      26890 non-null object
neighborhood_services_dist

The ward column contains records as "Ward n" and other as "n". In order to have a consistent data type, the "Ward n" record are converted into "n". Finally, the column data is converted into integer.

In [41]:
potholes_df['ward'] = potholes_df['ward'].str.extract('(\d+)',expand=False).astype(int)
potholes_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26976 entries, 0 to 29450
Data columns (total 25 columns):
CASE_ENQUIRY_ID                   26976 non-null int64
OPEN_DT                           26976 non-null datetime64[ns]
TARGET_DT                         26976 non-null datetime64[ns]
CLOSED_DT                         26648 non-null datetime64[ns]
OnTime_Status                     26976 non-null object
CASE_STATUS                       26976 non-null object
CLOSURE_REASON                    26976 non-null object
CASE_TITLE                        26976 non-null object
QUEUE                             26976 non-null object
Location                          26976 non-null object
fire_district                     26823 non-null float64
pwd_district                      26873 non-null object
city_council_district             26970 non-null float64
police_district                   26897 non-null object
neighborhood                      26890 non-null object
neighborhood_services_dist

### 4.3 The misterious ward "0"

Several records contains a ward value of 0. In order to decide is this value is meaningfull, let's plot the data on a map to see if the ward 0 records are grouped.

In [42]:
# Extracted records of interest
wardZero_df = potholes_df[potholes_df.ward==0]
defaultLocation = wardZero_df[["LATITUDE","LONGITUDE"]].drop_duplicates()
defaultLocation

,LATITUDE,LONGITUDE
93,42.3594,-71.0587


In [43]:
# Extract default location
defaultLocation_lst = [defaultLocation.LATITUDE[93],defaultLocation.LONGITUDE[93]]
defaultLocation_lst

[42.359400000000001, -71.058700000000002]

<font color='red'>**NOTE: As part of the goal of this capstone, the choice is made to experiment with a map plotting package instead of just retrieving the corresponding the location from the Web.**</font>

In [44]:
# The folium packages is used
%matplotlib inline
import folium

In [45]:
map_ward_0 = folium.Map(location=defaultLocation_lst,zoom_start=14)

folium.Marker(location=defaultLocation_lst, 
              popup='Ward 0',
              icon=folium.Icon(color='red',icon='info-sign',)).add_to(map_ward_0)

map_ward_0

In [46]:
# Extract the Location feature of the potholes assigned to the Ward 0.
print(wardZero_df.shape)
for loc in wardZero_df.Location.drop_duplicates():
    print(loc)

(37, 25)
INTERSECTION of N Washington St & Rutherford Ave  Boston  MA
INTERSECTION of Summit St & Metropolitan Ave  Boston  MA
INTERSECTION of Saint Pauls Ave & Pond St  Boston  MA
INTERSECTION of Saint Mary's St & Beacon St  Boston  MA
INTERSECTION of Mountfort St & Saint Mary's St  Boston  MA
INTERSECTION of Medfield St & Saint Mary's St  Boston  MA
INTERSECTION of Cambridge St & Crescent St  Charlestown  MA
INTERSECTION of Commonwealth Ave & University Rd  Boston  MA


Later on the Google geo-API will be used to correct some of the locations. In order to avoid wastin an unecessary number of query (limit per day). The locations above are searched using Google Maps and the Longitude and Latitude are retrieved.

In [47]:
# Prepare dictionaries mapping the locations to their longitudes, latitues, and zip.
Ward_0_Loc_Lat = {"INTERSECTION of N Washington St & Rutherford Ave  Boston  MA":42.371427,
                 "INTERSECTION of Summit St & Metropolitan Ave  Boston  MA":42.252180,
                 "INTERSECTION of Saint Pauls Ave & Pond St  Boston  MA":42.309236,
                 "INTERSECTION of Saint Mary's St & Beacon St  Boston  MA":42.346021,
                 "INTERSECTION of Mountfort St & Saint Mary's St  Boston  MA":42.348591,
                 "INTERSECTION of Medfield St & Saint Mary's St  Boston  MA":42.345236,
                 "INTERSECTION of Cambridge St & Crescent St  Charlestown  MA":42.382121,
                 "INTERSECTION of Commonwealth Ave & University Rd  Boston  MA":42.350490}
Ward_0_Loc_Lon = {"INTERSECTION of N Washington St & Rutherford Ave  Boston  MA":-71.06282,
                 "INTERSECTION of Summit St & Metropolitan Ave  Boston  MA":-71.109310,
                 "INTERSECTION of Saint Pauls Ave & Pond St  Boston  MA":-71.134461,
                 "INTERSECTION of Saint Mary's St & Beacon St  Boston  MA":-71.106668,
                 "INTERSECTION of Mountfort St & Saint Mary's St  Boston  MA":-71.106880,
                 "INTERSECTION of Medfield St & Saint Mary's St  Boston  MA":-71.106298,
                 "INTERSECTION of Cambridge St & Crescent St  Charlestown  MA":-71.080858,
                 "INTERSECTION of Commonwealth Ave & University Rd  Boston  MA":-71.109661}
Ward_0_Loc_Zip = {"INTERSECTION of N Washington St & Rutherford Ave  Boston  MA":2129,
                 "INTERSECTION of Summit St & Metropolitan Ave  Boston  MA":2136,
                 "INTERSECTION of Saint Pauls Ave & Pond St  Boston  MA":2130,
                 "INTERSECTION of Saint Mary's St & Beacon St  Boston  MA":2215,
                 "INTERSECTION of Mountfort St & Saint Mary's St  Boston  MA":2446,
                 "INTERSECTION of Medfield St & Saint Mary's St  Boston  MA":2215,
                 "INTERSECTION of Cambridge St & Crescent St  Charlestown  MA":2129,
                 "INTERSECTION of Commonwealth Ave & University Rd  Boston  MA":2215}

# Create dataframe containing the corrected values related to Ward 0.
Ward_0_corrected_df = pd.DataFrame({"LATITUDE":Ward_0_Loc_Lat,
                                    "LONGITUDE":Ward_0_Loc_Lon,
                                    "LOCATION_ZIPCODE":Ward_0_Loc_Zip})

Ward_0_corrected_df['Location'] = Ward_0_Loc_Zip.keys()
Ward_0_corrected_df.reset_index(inplace=True,drop=True)
Ward_0_corrected_df.head()

,LATITUDE,LOCATION_ZIPCODE,LONGITUDE,Location
0,42.382121,2129,-71.080858,INTERSECTION of N Washington St & Rutherford A...
1,42.350490,2215,-71.109661,INTERSECTION of Summit St & Metropolitan Ave ...
2,42.345236,2215,-71.106298,INTERSECTION of Saint Pauls Ave & Pond St Bos...
3,42.348591,2446,-71.106880,INTERSECTION of Saint Mary's St & Beacon St B...
4,42.371427,2129,-71.062820,INTERSECTION of Mountfort St & Saint Mary's St...


In [48]:
# Apply the merge:
potholes_df_clone = potholes_df[potholes_df.ward==0].copy()
potholes_df_clone=potholes_df_clone.merge(right=Ward_0_corrected_df,on='Location')

# Re-organize and rename the column to keep only the data from the mapping.
potholes_df_clone.drop(["LOCATION_ZIPCODE_x","LATITUDE_x","LONGITUDE_x"],inplace=True,axis=1)
potholes_df_clone.rename(index=str,columns={"LATITUDE_y":"LATITUDE",
                                           "LOCATION_ZIPCODE_y":"LOCATION_ZIPCODE",
                                           "LONGITUDE_y":"LONGITUDE"},inplace=True)

potholes_df_No_Ward0 = potholes_df[potholes_df.ward!=0]

potholes_df = pd.concat([potholes_df_No_Ward0,potholes_df_clone])


In [49]:
potholes_df[potholes_df.ward==0].head()

,CASE_ENQUIRY_ID,CASE_STATUS,CASE_TITLE,CLOSED_DT,CLOSURE_REASON,ClosedPhoto_Bool,LATITUDE,LOCATION_STREET_NAME,LOCATION_ZIPCODE,LONGITUDE,...,SubmittedPhoto_Bool,TARGET_DT,city_council_district,fire_district,neighborhood,neighborhood_services_district,police_district,precinct,pwd_district,ward
0,101002062057,Closed,Request for Pothole Repair,2017-04-13 12:30:30,case closed. closed date : 2017-04-13 12:30:30...,True,42.382121,INTERSECTION N Washington St & Rutherford Ave,2129.0,-71.080858,...,False,2017-04-13 08:30:00,0.0,3.0,NaN,0.0,NaN,NaN,NaN,0
1,101002064025,Closed,Request for Pothole Repair,2017-04-18 10:27:17,case closed. closed date : 2017-04-18 10:27:17...,True,42.382121,INTERSECTION N Washington St & Rutherford Ave,2129.0,-71.080858,...,False,2017-04-14 16:39:47,0.0,3.0,NaN,0.0,NaN,NaN,NaN,0
2,101002066010,Closed,Request for Pothole Repair,2017-04-18 10:26:24,case closed. closed date : 2017-04-18 10:26:24...,True,42.382121,INTERSECTION N Washington St & Rutherford Ave,2129.0,-71.080858,...,False,2017-04-19 08:30:00,0.0,3.0,NaN,0.0,NaN,NaN,NaN,0
3,101002066592,Closed,Request for Pothole Repair,2017-04-18 10:26:08,case closed. closed date : 2017-04-18 10:26:08...,True,42.382121,INTERSECTION N Washington St & Rutherford Ave,2129.0,-71.080858,...,False,2017-04-19 08:31:05,0.0,3.0,NaN,0.0,NaN,NaN,NaN,0
4,101002012963,Closed,Request for Pothole Repair,2017-02-14 11:44:44,case closed. closed date : 2017-02-14 11:44:44...,False,42.382121,INTERSECTION N Washington St & Rutherford Ave,2129.0,-71.080858,...,True,2017-02-15 11:44:42,0.0,3.0,NaN,0.0,NaN,NaN,NaN,0


In [50]:
potholes_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26976 entries, 0 to 36
Data columns (total 25 columns):
CASE_ENQUIRY_ID                   26976 non-null int64
CASE_STATUS                       26976 non-null object
CASE_TITLE                        26976 non-null object
CLOSED_DT                         26648 non-null datetime64[ns]
CLOSURE_REASON                    26976 non-null object
ClosedPhoto_Bool                  26976 non-null bool
LATITUDE                          26976 non-null float64
LOCATION_STREET_NAME              26976 non-null object
LOCATION_ZIPCODE                  16944 non-null float64
LONGITUDE                         26976 non-null float64
Location                          26976 non-null object
OPEN_DT                           26976 non-null datetime64[ns]
OnTime_Status                     26976 non-null object
QUEUE                             26976 non-null object
Source                            26976 non-null object
SubmittedPhoto_Bool               26976 non

### 4.3 Missing CLOSED_DT

As shown above, a number of potholes have not been assigned a closed date. Since the purpose of this project is to analyze the response time of the city to repair potholes, these cases are not relevant. Therefore, they are removed from the data frame.

In [51]:
potholes_df=potholes_df[potholes_df.CLOSED_DT.notnull()]
potholes_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26648 entries, 0 to 36
Data columns (total 25 columns):
CASE_ENQUIRY_ID                   26648 non-null int64
CASE_STATUS                       26648 non-null object
CASE_TITLE                        26648 non-null object
CLOSED_DT                         26648 non-null datetime64[ns]
CLOSURE_REASON                    26648 non-null object
ClosedPhoto_Bool                  26648 non-null bool
LATITUDE                          26648 non-null float64
LOCATION_STREET_NAME              26648 non-null object
LOCATION_ZIPCODE                  16754 non-null float64
LONGITUDE                         26648 non-null float64
Location                          26648 non-null object
OPEN_DT                           26648 non-null datetime64[ns]
OnTime_Status                     26648 non-null object
QUEUE                             26648 non-null object
Source                            26648 non-null object
SubmittedPhoto_Bool               26648 non

## 5. The default location

After inspecting the resulting data, it seems that a large number of potholes are located at the "Default" location corresponding to LATITUDE=42.3594, LONGITUDE=-71.0587.

In [52]:
defaultLocation_df = potholes_df[(potholes_df.LATITUDE==42.3594) &
                                 (potholes_df.LONGITUDE==-71.0587)]
defaultLocation_df.shape

(5353, 25)

In [53]:
# The default location Latitude and Longitude are replaced with NaN in order to facilitate the cleaning.
potholes_df.loc[(potholes_df.LATITUDE==42.3594) &
            (potholes_df.LONGITUDE==-71.0587),["LATITUDE","LONGITUDE"]] = np.NaN

In order to estimate the location of these potholes, a step by step process is chosen. The records corresponding to this default location are inspected. If sufficient information can be retrieve from other feature to estimate the location then the record is kept.

** Search by zip code**

In [54]:
# Identify the mapping between neighborhoods and zip codes
potholes_df[['neighborhood',"LOCATION_ZIPCODE"]].drop_duplicates().sort_values(by="LOCATION_ZIPCODE")

,neighborhood,LOCATION_ZIPCODE
456,Beacon Hill,2108.0
13,Downtown / Financial District,2108.0
6446,Back Bay,2108.0
1185,Boston,2108.0
22,Boston,2109.0
49,Downtown / Financial District,2109.0
5454,South Boston / South Boston Waterfront,2110.0
1420,Boston,2110.0
217,Downtown / Financial District,2110.0
451,Downtown / Financial District,2111.0


The mapping is not unique for most cases. However, the following zip/neighborhood are:
 - Charlestown: zip 2129
 - East Boston: zip 2128
 
 However, this would only help to fix three records.


The precinct feature are dropped is not relevant as the zones can be redefined every couple years.

In [55]:
potholes_df.drop(["precinct"],inplace=True,axis=1)
potholes_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26648 entries, 0 to 36
Data columns (total 24 columns):
CASE_ENQUIRY_ID                   26648 non-null int64
CASE_STATUS                       26648 non-null object
CASE_TITLE                        26648 non-null object
CLOSED_DT                         26648 non-null datetime64[ns]
CLOSURE_REASON                    26648 non-null object
ClosedPhoto_Bool                  26648 non-null bool
LATITUDE                          21295 non-null float64
LOCATION_STREET_NAME              26648 non-null object
LOCATION_ZIPCODE                  16754 non-null float64
LONGITUDE                         21295 non-null float64
Location                          26648 non-null object
OPEN_DT                           26648 non-null datetime64[ns]
OnTime_Status                     26648 non-null object
QUEUE                             26648 non-null object
Source                            26648 non-null object
SubmittedPhoto_Bool               26648 non

First let's take care of the single missing "OnTime_Status".

Create a new feature *OnTime_Status_Bool* which is True is OnTime_Status is "ONTIME".

In [56]:
potholes_df["OnTime_Status_Bool"] = potholes_df["OnTime_Status"]=="ONTIME"
potholes_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26648 entries, 0 to 36
Data columns (total 25 columns):
CASE_ENQUIRY_ID                   26648 non-null int64
CASE_STATUS                       26648 non-null object
CASE_TITLE                        26648 non-null object
CLOSED_DT                         26648 non-null datetime64[ns]
CLOSURE_REASON                    26648 non-null object
ClosedPhoto_Bool                  26648 non-null bool
LATITUDE                          21295 non-null float64
LOCATION_STREET_NAME              26648 non-null object
LOCATION_ZIPCODE                  16754 non-null float64
LONGITUDE                         21295 non-null float64
Location                          26648 non-null object
OPEN_DT                           26648 non-null datetime64[ns]
OnTime_Status                     26648 non-null object
QUEUE                             26648 non-null object
Source                            26648 non-null object
SubmittedPhoto_Bool               26648 non

In [57]:
# Construct a new feature: Duration in days between the date of the claim and the repair
potholes_df['time_repair'] = (potholes_df.CLOSED_DT-potholes_df.OPEN_DT)/ np.timedelta64(1, 'D')
potholes_df['time_repair'].describe()

count    26648.000000
mean         2.568061
std         14.805684
min         -0.000602
25%          0.002917
50%          0.200463
75%          1.150787
max        561.853229
Name: time_repair, dtype: float64

The following choices are made:
- Delete cases if the delay is negative
- Delete cases if delay is geater than six months (delay due to wether, construction,...)

In [58]:
potholes_df = potholes_df[(potholes_df.time_repair>=0) & (potholes_df.time_repair<=180)]
potholes_df.shape

(26431, 26)

### 5. Default location and incorrect address

#### 5.1. Missing location 

The next big challenge of the cleaning process is to correct the records with incorrect addresses. By default, their longitude and latitude were assigned to the default location (see previous). The idea is to create a function that takes a row as an input, creates an approximate addres, and retrieves the longitude and latitude from Google's geo-API.

The main challenge with this method is the limit of the API. In order to deal with it in an efficient way, we will extract all the locations from the data frame and save the unique values in a csv file. Using a separate Notebook, an algorithm is implemented to retrieve as much information as possible.  
**PLEASE SEE 02b_Google_Geo_API_Fetcher.ipynb**

In [59]:
missing_zip_lat_long_df = potholes_df[(potholes_df.LOCATION_ZIPCODE.isnull())|
                                   (potholes_df.LATITUDE.isnull())|
                                   (potholes_df.LONGITUDE.isnull())]
missing_zip_lat_long_df =missing_zip_lat_long_df[["Location"]].drop_duplicates()
missing_zip_lat_long_df.sort_values(by="Location",ascending=True,inplace=True)
missing_zip_lat_long_df.reset_index(inplace=True,drop=True)
missing_zip_lat_long_df.shape

(4513, 1)

In [60]:
missing_zip_lat_long_df.to_csv("./Intermediate Data/Missing_Zip_Lat_Long.csv")

#### 5.2. Analysis of the retrieved locations
The purpose of this section is to investigat the quality of the data retrieved using Google geo-API. Since the API returns the best match based on the input address, there is a chance that the information might be inaccurate.

**Data import**

In [61]:
# Load data as a Pandas dataframe.
# Note the csv does not have header title, they are set using Pandas
full_name = "./Intermediate Data/Retrieved_Zip_Lat_Long.csv"
retrieved_locations_df = pd.read_csv(full_name,header=None,
                                     names=["location","LATITUDE","LONGITUDE","LOCATION_ZIPCODE"],
                                     na_values=["None","ERROR"],index_col=0)

In [62]:
# Inspect content
print(retrieved_locations_df.shape)
print(retrieved_locations_df.columns)
retrieved_locations_df.head()

(4531, 4)
Index(['location', 'LATITUDE', 'LONGITUDE', 'LOCATION_ZIPCODE'], dtype='object')


,location,LATITUDE,LONGITUDE,LOCATION_ZIPCODE
0,Adams St Dorchester MA 02124,42.282604,-71.055732,2124.0
1,Alabama St Mattapan MA 02126,42.275567,-71.102592,2126.0
2,Albany St Boston MA 02111,42.349538,-71.060012,2111.0
3,Alford St Charlestown MA 02129,42.389409,-71.071097,2129.0
4,Amory St Roxbury MA 02119,42.319729,-71.099914,2119.0


In [63]:
retrieved_locations_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4531 entries, 0 to 4529
Data columns (total 4 columns):
location            4531 non-null object
LATITUDE            4531 non-null float64
LONGITUDE           4531 non-null float64
LOCATION_ZIPCODE    4526 non-null float64
dtypes: float64(3), object(1)
memory usage: 177.0+ KB


In [64]:
# Check the range of values for the longitude and latitude of the retrieved locations
print("Latitude: min",retrieved_locations_df.LATITUDE.min(axis=0))
print("Latitude: max",retrieved_locations_df.LATITUDE.max(axis=0))
print("Longitude: min",retrieved_locations_df.LONGITUDE.min(axis=0))
print("Longitude: max",retrieved_locations_df.LONGITUDE.max(axis=0))

Latitude: min 41.5172543
Latitude: max 42.9017407
Longitude: min -71.4872635
Longitude: max -70.9166865


The range of value corresponds to the area we are interested in.

In [65]:
retrieved_locations_df.head()

,location,LATITUDE,LONGITUDE,LOCATION_ZIPCODE
0,Adams St Dorchester MA 02124,42.282604,-71.055732,2124.0
1,Alabama St Mattapan MA 02126,42.275567,-71.102592,2126.0
2,Albany St Boston MA 02111,42.349538,-71.060012,2111.0
3,Alford St Charlestown MA 02129,42.389409,-71.071097,2129.0
4,Amory St Roxbury MA 02119,42.319729,-71.099914,2119.0


**Data quality**

As shown above, 5 locations could not be matched to a zip code.

In [66]:
missing_zip_df = retrieved_locations_df[retrieved_locations_df.LOCATION_ZIPCODE.isnull()]
missing_zip_df

,location,LATITUDE,LONGITUDE,LOCATION_ZIPCODE
7,Atlantic Ave Boston MA 02108,42.353221,-71.054551,NaN
130,Seaver St Dorchester MA 02121,42.307094,-71.088105,NaN
149,W Fourth St South Boston MA 02127,42.339230,-71.053401,NaN
2388,INTERSECTION of Essex St & Interstate 93 S Bo...,42.356405,-71.051220,NaN
3335,INTERSECTION of Massachusetts Tpke E & Hunting...,42.347896,-71.079681,NaN


We have two options, either drop the record or try to identify an error that lead the API to return a null value for the zip.
First, let's plot these points to see if the coordinates make sense.

In [67]:
# List comprehension to ontain the coordinates of the missing locations as list of lists.
missing_zip_lst = [[x,y] for x,y in 
                         zip(missing_zip_df.LATITUDE.values,missing_zip_df.LONGITUDE.values)]

# Extract the missing location description so they can be added to the markers.
missing_location_lst = list(missing_zip_df.location.values)

# Create map object
map_missing_zip = folium.Map(location=defaultLocation_lst,zoom_start=12)

# Start tracker to be used with missing_location_lst
missing_location_index=0

# Create markers and plot map
for loc in missing_zip_lst:
    folium.Marker(location=loc, popup=str(missing_location_lst[missing_location_index]),
              icon=folium.Icon(color='red',icon='info-sign',)).add_to(map_missing_zip)
    missing_location_index+=1
map_missing_zip

For simplicity, these five records are deleted. They represent a small fraction of the data set.

In [68]:
retrieved_locations_df = retrieved_locations_df[retrieved_locations_df.LOCATION_ZIPCODE.notnull()]
retrieved_locations_df.shape

(4526, 4)

**Data merge**  
The next step consists of mergin the retrieved data with the main potholes data frame. First we need to re-apply the string transformation that were used to facilitate the geo-search.

In [69]:
missing_zip_lat_long_df = potholes_df[(potholes_df.LOCATION_ZIPCODE.isnull())|
                                   (potholes_df.LATITUDE.isnull())|
                                   (potholes_df.LONGITUDE.isnull())].copy()

# If address starts with a 0, remove it.
locations = missing_zip_lat_long_df.loc[:,"Location"]

locations = locations.str.extract(r"^0* *(.*)$",expand=False)

# If address contains "-" then keep the second street number.
locations = locations.str.split(pat='-',n=1)
locations = locations.str[-1]
missing_zip_lat_long_df.loc[:,"Location"] = locations

print(missing_zip_lat_long_df.shape)
print(missing_zip_lat_long_df.columns)

(10835, 26)
Index(['CASE_ENQUIRY_ID', 'CASE_STATUS', 'CASE_TITLE', 'CLOSED_DT',
       'CLOSURE_REASON', 'ClosedPhoto_Bool', 'LATITUDE',
       'LOCATION_STREET_NAME', 'LOCATION_ZIPCODE', 'LONGITUDE', 'Location',
       'OPEN_DT', 'OnTime_Status', 'QUEUE', 'Source', 'SubmittedPhoto_Bool',
       'TARGET_DT', 'city_council_district', 'fire_district', 'neighborhood',
       'neighborhood_services_district', 'police_district', 'pwd_district',
       'ward', 'OnTime_Status_Bool', 'time_repair'],
      dtype='object')


In [70]:
# Merge the two data frames
missing_zip_lat_long_df=missing_zip_lat_long_df.merge(right=retrieved_locations_df,
                                                                  how='inner',left_on='Location',
                                                                  right_on='location')

In [71]:
missing_zip_lat_long_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10838 entries, 0 to 10837
Data columns (total 30 columns):
CASE_ENQUIRY_ID                   10838 non-null int64
CASE_STATUS                       10838 non-null object
CASE_TITLE                        10838 non-null object
CLOSED_DT                         10838 non-null datetime64[ns]
CLOSURE_REASON                    10838 non-null object
ClosedPhoto_Bool                  10838 non-null bool
LATITUDE_x                        5541 non-null float64
LOCATION_STREET_NAME              10838 non-null object
LOCATION_ZIPCODE_x                1004 non-null float64
LONGITUDE_x                       5541 non-null float64
Location                          10838 non-null object
OPEN_DT                           10838 non-null datetime64[ns]
OnTime_Status                     10838 non-null object
QUEUE                             10838 non-null object
Source                            10838 non-null object
SubmittedPhoto_Bool               1083

In [72]:
# Delete old comlumns
missing_zip_lat_long_df.drop(labels=["LATITUDE_x","LOCATION_ZIPCODE_x","LONGITUDE_x","location"],
                             inplace=True,
                             axis=1)
missing_zip_lat_long_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10838 entries, 0 to 10837
Data columns (total 26 columns):
CASE_ENQUIRY_ID                   10838 non-null int64
CASE_STATUS                       10838 non-null object
CASE_TITLE                        10838 non-null object
CLOSED_DT                         10838 non-null datetime64[ns]
CLOSURE_REASON                    10838 non-null object
ClosedPhoto_Bool                  10838 non-null bool
LOCATION_STREET_NAME              10838 non-null object
Location                          10838 non-null object
OPEN_DT                           10838 non-null datetime64[ns]
OnTime_Status                     10838 non-null object
QUEUE                             10838 non-null object
Source                            10838 non-null object
SubmittedPhoto_Bool               10838 non-null bool
TARGET_DT                         10838 non-null datetime64[ns]
city_council_district             10832 non-null float64
fire_district                  

In [73]:
# Rename column to concatenate the two main dataframes usign the same column names
missing_zip_lat_long_df.rename(index=str,columns={"LATITUDE_y":"LATITUDE",
                                "LONGITUDE_y":"LONGITUDE",
                                "LOCATION_ZIPCODE_y":"LOCATION_ZIPCODE"},inplace=True)
missing_zip_lat_long_df.columns

Index(['CASE_ENQUIRY_ID', 'CASE_STATUS', 'CASE_TITLE', 'CLOSED_DT',
       'CLOSURE_REASON', 'ClosedPhoto_Bool', 'LOCATION_STREET_NAME',
       'Location', 'OPEN_DT', 'OnTime_Status', 'QUEUE', 'Source',
       'SubmittedPhoto_Bool', 'TARGET_DT', 'city_council_district',
       'fire_district', 'neighborhood', 'neighborhood_services_district',
       'police_district', 'pwd_district', 'ward', 'OnTime_Status_Bool',
       'time_repair', 'LATITUDE', 'LONGITUDE', 'LOCATION_ZIPCODE'],
      dtype='object')

In [74]:
# Concatenate the two dataframes
potholes_no_missing_df = potholes_df[(potholes_df.LOCATION_ZIPCODE.notnull())&
                                   (potholes_df.LATITUDE.notnull()) &
                                   (potholes_df.LONGITUDE.notnull())].copy()
potholes_df = pd.concat([potholes_no_missing_df,missing_zip_lat_long_df])

In [75]:
potholes_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26434 entries, 0 to 10837
Data columns (total 26 columns):
CASE_ENQUIRY_ID                   26434 non-null int64
CASE_STATUS                       26434 non-null object
CASE_TITLE                        26434 non-null object
CLOSED_DT                         26434 non-null datetime64[ns]
CLOSURE_REASON                    26434 non-null object
ClosedPhoto_Bool                  26434 non-null bool
LATITUDE                          26434 non-null float64
LOCATION_STREET_NAME              26434 non-null object
LOCATION_ZIPCODE                  26434 non-null float64
LONGITUDE                         26434 non-null float64
Location                          26434 non-null object
OPEN_DT                           26434 non-null datetime64[ns]
OnTime_Status                     26434 non-null object
OnTime_Status_Bool                26434 non-null bool
QUEUE                             26434 non-null object
Source                            26434 no

In [76]:
potholes_df_clone = potholes_df.copy()
potholes_df_clone = potholes_df_clone[(potholes_df_clone.city_council_district.notnull()) &
                                      (potholes_df_clone.fire_district.notnull()) &
                                      (potholes_df_clone.neighborhood.notnull()) &
                                      (potholes_df_clone.neighborhood_services_district.notnull()) &
                                      (potholes_df_clone.police_district.notnull()) &
                                      (potholes_df_clone.pwd_district.notnull())]
potholes_df_clone.shape

(26195, 26)

If we decide to get rid of the missing data per row, we would lose ~250 records. This represent less than one percent of the data.

In [77]:
potholes_df_clone = potholes_df_clone[(potholes_df_clone.city_council_district.notnull()) &
                                      (potholes_df_clone.fire_district.notnull()) &
                                      (potholes_df_clone.neighborhood.notnull()) &
                                      (potholes_df_clone.neighborhood_services_district.notnull()) &
                                      (potholes_df_clone.police_district.notnull()) &
                                      (potholes_df_clone.pwd_district.notnull())]

## 6. Intersections

If a record contains the word "INTERSECTION" in the location string, we assign a True value to the new feature is_intersection, else we fill it with False.

In [ ]:
potholes_df_clone['is_intersction'] = potholes_df_clone.Location.str.contains("INTERSECTION")
potholes_df_clone['is_intersction'].describe()

## 7. Chestnut Hill

As previously stated, a small portion of the Chestnut Hill neighborhood (zip 02467) is located under the juridiction of the city of Boston. In order to understand what data we possess, we must plot the potholes located in the zipcode 02467 on top of the neighborhood boundaries.

In [90]:
# List comprehension to ontain the coordinates of the missing locations as list of lists.
chestnut_hill_df = potholes_df[potholes_df.LOCATION_ZIPCODE==2467] 
chestnut_hill_lst = [[x,y] for x,y in 
                         zip(chestnut_hill_df.LATITUDE.values,chestnut_hill_df.LONGITUDE.values)]

# Create map object
chestnut_hill_map = folium.Map(location=defaultLocation_lst,zoom_start=12)

# Create markers and plot map
for loc in chestnut_hill_lst:
    folium.Marker(location=loc, popup='?',
              icon=folium.Icon(color='red',icon='info-sign',)).add_to(chestnut_hill_map)
    
#chestnut_hill_map.choropleth(geo_path='./Original Data/Map per zip/Zip_Codes.json',
#                             style_function=lambda x: {'fillColor':'green' if x['properties']['ZIP5'] != "2467" 
#                                                       else 'orange'})

chestnut_hill_map.add_child(folium.GeoJson(data=open('./Original Data/Map per zip/map.geojson'),
                                          name='Zip codes',
                                           style_function=lambda x: {'fillColor':'white' if x['properties']['ZIP5'] != "02467" 
                                                                                     else 'red'}))

chestnut_hill_map

As shown above, the potholes located in Chestnut Hill are located within the boundary of Boston. This will be accounted for during the main analysis.

## 8. Output file

In [91]:
potholes_df_clone.to_csv("./Cleaned Data/Closed_Pothole_Cases_Cleaned.csv")